Main Body Architecture Consists of 2 parts
    1. Convolutional Layers - extract useful features
    2. Fully Connected (FC) Layers  - using features to classify images

Use YoloV10's backbone as the pretrained model for first part of convolutional layer. 
- Choose not to freeeze the weights as part of the fine tuning process (might freeze due to time)
- Utilize our own custom made training function due to added layers

Initalize Pytorch

In [4]:
import torch
#verify torch is installed
print(torch.__version__)
import torch.nn as nn 
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
import torch.optim as optim  

#set random seed 
torch.manual_seed(1)

ModuleNotFoundError: No module named 'torch'